<a href="https://colab.research.google.com/github/saratbhargava/Deeplearning-with-python/blob/master/Chp7/Chp7.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some tasks require more than one type of data like text data, images and metadata. All of these requires three different types of neural nets. Text data requires RNN or 1D CNN. Image data requires CNN.

# Imports

In [9]:
import os

import numpy as np
import pandas as pd

In [40]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, Flatten, MaxPooling2D, MaxPooling1D
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import MSE, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import MAE
from tensorflow.keras.layers import Embedding, LSTM, concatenate, GlobalMaxPooling1D

In [4]:
input_tensor = Input(shape=(32,))
output = Dense(32, activation=relu)(input_tensor)
model = Model(input_tensor, output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [5]:
input_tensor = Input(shape=(32,))
layer1_output = Dense(32, activation=relu)(input_tensor)
layer2_output = Dense(16, activation=relu)(layer1_output)
output = Dense(8, activation=relu)(layer2_output)
model = Model(input_tensor, output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
Total params: 1,720
Trainable params: 1,720
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer=Adam(), loss='MSE', metrics=['MAE'])

x_train = np.random.rand(100, 32)
y_train = np.random.rand(100, 8)

model.fit(x_train, y_train, batch_size=32, epochs=10)

print(model.evaluate(x_train, y_train))

Epoch 1/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1707 - MAE: 0.3388
Epoch 2/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1681 - MAE: 0.3350
Epoch 3/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1657 - MAE: 0.3315
Epoch 4/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1647 - MAE: 0.3300
Epoch 5/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1634 - MAE: 0.3277
Epoch 6/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1614 - MAE: 0.3258
Epoch 7/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1605 - MAE: 0.3257
Epoch 8/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1594 - MAE: 0.3247
Epoch 9/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1573 - MAE: 0.3222
Epoch 10/10
4/4 [==============================] - 0s 2ms/step - loss: 0.1522 - MAE: 0.3173
[0.15220408141613007, 0.3172810971736908]


In [15]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

In [24]:
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = Embedding(64, text_vocabulary_size)(text_input)
encoded_text = LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = Embedding(32, question_vocabulary_size)(question_input)
encoded_question = LSTM(16)(embedded_question)

concat = concatenate([encoded_text, encoded_question], axis=-1)
answer = Dense(answer_vocabulary_size, activation='softmax')(concat)

In [34]:
model = Model([text_input, question_input], answer)
model.compile(optimizer=Adam(), loss='categorical_crossentropy',
              metrics='acc')

In [35]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
____________________________________________________________________________________________

In [ ]:
import numpy as np

num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size,
size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size,
                          size=(num_samples, max_length))
answers = np.random.randint(0, 1,
                          size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)
model.fit({'text': text, 'question': question}, answers,
                  epochs=10, batch_size=128)

In [65]:
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(500,), dtype='int32', name='posts')
embedded_posts = Embedding(vocabulary_size, 256)(posts_input)
x = Conv1D(128, 5, activation='relu')(embedded_posts)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)

age_prediction = Dense(1, name='age')(x)
income_prediction = Dense(num_income_groups, activation='softmax',
                                 name='income')(x)
gender_prediction = Dense(1, activation='sigmoid', name='gender')(x)

model = Model(
    posts_input, [age_prediction, income_prediction, gender_prediction])

In [66]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

In [68]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 500, 256)     12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 496, 128)     163968      embedding_12[0][0]               
__________________________________________________________________________________________________
max_pooling1d_14 (MaxPooling1D) (None, 99, 128)      0           conv1d_34[0][0]                  
___________________________________________________________________________________________